In [1]:
import pandas as pd
import numpy as np

!pip install catboost >> none
import catboost
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings('ignore')

## Препроцессинг

In [2]:
df = pd.read_csv('train_dataset_train (1).csv')

In [3]:
pd.set_option('display.max_columns', None)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629.0,Участок,7503.0,N,0.0,Y,N,87.0,5.0,1.0,0.0,2.0,0.00,0.0,"Fishhook,USB",41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976.0,ММПО,7503.0,N,0.0,N,N,107.0,5.0,1.0,0.0,2.0,0.00,0.0,"screen protector,Case(Q613B),case(208B40-DB)",68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962.0,Цех,7503.0,N,0.0,Y,N,50.0,5.0,1.0,0.0,2.0,0.00,0.0,"Pendant Necklaces,Rings for Women,Necklaces",56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964.0,Цех,7503.0,N,0.0,Y,N,416.0,5.0,1.0,0.0,2.0,35.34,0.0,Motorcycle Signal Lamp,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819.0,Участок,7503.0,N,0.0,Y,N,795.0,5.0,1.0,0.0,2.0,52.52,0.0,backpack,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,1022_-1,102976.0,ММПО,7503.0,N,0.0,N,N,25.0,5.0,0.0,0.0,2.0,0.00,0.0,KEY CHAIN,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,0
5999996,2234489,1022_-1,241963.0,Цех,7503.0,N,0.0,Y,N,83.0,5.0,1.0,0.0,2.0,0.00,0.0,Dolls,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,0
5999997,4304572,1041_-1,102971.0,Цех,7506.0,N,0.0,N,N,1700.0,5.0,1.0,0.0,2.0,94.09,0.0,Down jacket,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,0
5999998,6550634,1018_-1,102152.0,Цех,7506.0,N,0.0,N,N,269.0,5.0,1.0,0.0,2.0,0.00,0.0,0,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,0


In [4]:
test = pd.read_csv('test_dataset_test.csv')

In [5]:
df.index_oper = df.index_oper.astype(str)

dfstr = df.loc[df.index_oper.str.slice(-2,) == '.0']
dfstr1 = df.loc[(df.index_oper.str.slice(-2,) != '.0')]

dfstr1.index_oper.replace(' ', np.nan, inplace=True)
dfstr1.index_oper = dfstr1.index_oper.fillna('102976')

dfstr.index_oper = dfstr.index_oper.astype(float)
dfstr1.index_oper = dfstr1.index_oper.astype(int)
df = pd.concat([dfstr, dfstr1], axis=0)

df.index_oper = df.index_oper.astype(int)

df = df.drop(['mailrank',
              'postmark',
              'priority',
              'mailtype',
              'directctg',
              'name_mfi',
              'oper_type + oper_attr'], axis=1)

cat_cols = ['type',
            'is_privatecategory',
            'is_in_yandex',
            'is_return']

target_col = df.label

In [6]:
test.index_oper = test.index_oper.astype(str)

teststr = test.loc[test.index_oper.str.slice(-2,) == '.0']
teststr1 = test.loc[(test.index_oper.str.slice(-2,) != '.0')]

teststr1.index_oper.replace(' ', np.nan, inplace=True)
teststr1.index_oper = teststr1.index_oper.fillna('102976')

teststr.index_oper = teststr.index_oper.astype(float)
teststr1.index_oper = teststr1.index_oper.astype(int)
test = pd.concat([teststr, teststr1], axis=0)

test.index_oper = test.index_oper.astype(int)

test = test.drop(['mailrank',
                  'postmark',
                  'priority',
                  'mailtype',
                  'directctg',
                  'name_mfi',
                  'oper_type + oper_attr'], axis=1)

In [7]:
df.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

,id,index_oper,class,weight,mailctg,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
id,1.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00
index_oper,0.00,1.00,0.23,0.04,0.08,0.07,-0.00,-0.02,-0.48,-0.60,-0.36,-0.50,-0.52,0.02,0.09,-0.10,0.01,-0.03
class,-0.00,0.23,1.00,0.00,0.01,0.00,-0.01,-0.01,-0.28,-0.35,-0.20,-0.28,-0.34,0.02,0.04,0.00,0.02,0.12
weight,0.00,0.04,0.00,1.00,0.22,0.59,0.35,0.10,-0.06,-0.05,-0.05,-0.06,-0.04,-0.02,0.11,-0.14,-0.02,-0.04
mailctg,-0.00,0.08,0.01,0.22,1.00,0.27,0.18,0.08,-0.09,-0.10,-0.07,-0.09,-0.09,-0.07,0.08,-0.56,-0.06,-0.15
transport_pay,-0.00,0.07,0.00,0.59,0.27,1.00,0.17,-0.02,-0.08,-0.09,-0.05,-0.08,-0.06,-0.03,0.16,-0.17,-0.02,-0.06
weight_mfi,0.00,-0.00,-0.01,0.35,0.18,0.17,1.00,0.41,-0.06,0.00,-0.06,-0.04,-0.04,-0.03,-0.10,-0.13,-0.02,-0.03
price_mfi,0.00,-0.02,-0.01,0.10,0.08,-0.02,0.41,1.00,-0.01,0.02,-0.02,-0.01,-0.01,-0.01,-0.05,-0.05,-0.01,-0.01
dist_qty_oper_login_1,0.00,-0.48,-0.28,-0.06,-0.09,-0.08,-0.06,-0.01,1.00,0.69,0.84,0.89,0.80,-0.01,-0.05,0.12,0.00,0.13
total_qty_oper_login_1,0.00,-0.60,-0.35,-0.05,-0.10,-0.09,0.00,0.02,0.69,1.00,0.53,0.78,0.71,-0.01,-0.10,0.11,0.00,0.06


In [8]:
df.sample(10)

,id,index_oper,type,is_privatecategory,class,is_in_yandex,is_return,weight,mailctg,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
3742693,6747847,102976,ММПО,N,0.0,N,N,101.0,1.0,0.00,300.0,1000.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,1,0,0,1
4116115,7059632,102002,Цех,N,0.0,N,N,349.0,1.0,0.00,0.0,0.0,153.0,26432806.0,6158583.0,32591389.0,58539804.0,0,1,0,0,0
735132,2613617,102968,ТИ,N,0.0,N,N,516.0,1.0,0.00,334.0,300.0,50.0,6455803.0,1423603.0,7879406.0,8340614.0,0,0,0,0,0
1164484,9655746,102971,Цех,N,0.0,N,N,31.0,1.0,0.00,30.0,250.0,184.0,67541214.0,7870.0,67549084.0,72981134.0,0,1,0,0,0
2129714,2076609,102976,ММПО,N,0.0,N,N,579.0,1.0,0.00,523.0,1000.0,972.0,68766046.0,91123247.0,159889293.0,164927295.0,0,0,0,0,0
1103963,2781372,102976,ММПО,N,0.0,N,N,37.0,1.0,0.00,38.0,700.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
521115,3571791,394963,Участок,N,0.0,Y,N,64.0,1.0,0.00,45.0,400.0,61.0,1950677.0,327735.0,2278412.0,9632467.0,0,0,0,0,0
3316320,1918130,102971,Цех,N,0.0,N,N,23.0,0.0,0.00,21.0,22.0,153.0,89268735.0,6799759.0,96068494.0,101200378.0,0,0,1,0,0
1271642,2579325,102971,Цех,N,0.0,N,N,174.0,1.0,24.11,16.0,50.0,172.0,48772076.0,11990.0,48784066.0,58042096.0,0,0,0,0,0
2091307,2521981,367962,Цех,N,0.0,Y,N,105.0,1.0,0.00,91.0,422.0,31.0,1384588.0,3401.0,1387989.0,10402044.0,0,0,0,0,0


In [9]:
target_col.value_counts()

0    5829020
1     170980
Name: label, dtype: int64

In [10]:
# Отношение количества значений большего класса к меньшему

weight = (target_col[target_col == 0].count()) / (target_col[target_col == 1].count())
weight

34.09182360510001

## Обучение модели

In [11]:
df = df.drop('label', axis=1)

In [14]:
cat = CatBoostClassifier(cat_features=cat_cols,
                         scale_pos_weight=weight,
                         learning_rate = 0.02,
                         l2_leaf_reg = 2,
                         depth = 8,
                         iterations=1000,
                         verbose=100)

In [15]:
cat.fit(df, target_col)

0:	learn: 0.6682122	total: 3.57s	remaining: 59m 29s
100:	learn: 0.2947791	total: 4m 46s	remaining: 42m 27s
200:	learn: 0.2801178	total: 9m 37s	remaining: 38m 16s
300:	learn: 0.2750920	total: 14m 24s	remaining: 33m 27s
400:	learn: 0.2727276	total: 19m 5s	remaining: 28m 31s
500:	learn: 0.2712640	total: 23m 46s	remaining: 23m 41s
600:	learn: 0.2702127	total: 28m 24s	remaining: 18m 51s
700:	learn: 0.2694110	total: 32m 57s	remaining: 14m 3s
800:	learn: 0.2685077	total: 37m 37s	remaining: 9m 20s
900:	learn: 0.2676866	total: 42m 26s	remaining: 4m 39s
999:	learn: 0.2669212	total: 47m 14s	remaining: 0us


In [16]:
prediction = cat.predict(test)

In [17]:
sub = pd.DataFrame(test['id'])
sub['label'] = prediction
sub.to_csv('ans.csv', sep=",", index=False, line_terminator='\n')